In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=Warning)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<center style="font-family:verdana;"><h1 style="font-size:200%; padding: 10px; background: #001f3f;"><b style="color:orange;">Pycaret</b></h1></center>

"PyCaret is an open-source, low-code machine learning library in Python that aims to reduce the cycle time from hypothesis to insights. It is well suited for seasoned data scientists who want to increase the productivity of their ML experiments by using PyCaret in their workflows or for citizen data scientists and those new to data science with little or no background in coding. PyCaret allows you to go from preparing your data to deploying your model within seconds using your choice of notebook environment."

https://pycaret.org/guide/

https://github.com/pycaret/pycaret/blob/master/examples/PyCaret%202%20Regression.ipynb

![](https://i1.wp.com/pycaret.org/wp-content/uploads/2020/04/thumbnail.png?fit=1166%2C656&ssl=1)pycaret.org

In [ ]:
df = pd.read_csv("/kaggle/input/learnplatform-covid19-impact-on-digital-learning/engagement_data/5802.csv", low_memory=False)
print(df.shape)
df.head()

In [ ]:
#Code by HarshGupta57 https://www.kaggle.com/harshgupta57/freediving-world-records-eda/notebook

import datetime

In [ ]:
#Code by HarshGupta57 https://www.kaggle.com/harshgupta57/freediving-world-records-eda/notebook

df[['year','month','date']] = df['time'].apply(lambda x : pd.Series(str(x).split('-')))

In [ ]:
#Code by HarshGupta57 https://www.kaggle.com/harshgupta57/freediving-world-records-eda/notebook

df.drop('time', inplace=True, axis =1)

In [ ]:
df.head()

In [ ]:
#Code by Parul Pandey  https://www.kaggle.com/parulpandey/a-guide-to-handling-missing-values-in-python


from sklearn.impute import SimpleImputer
df_most_frequent = df.copy()
#setting strategy to 'mean' to impute by the mean
mean_imputer = SimpleImputer(strategy='most_frequent')# strategy can also be mean or median 
df_most_frequent.iloc[:,:] = mean_imputer.fit_transform(df_most_frequent)

In [ ]:
df_most_frequent.isnull().sum()

In [ ]:
from sklearn.preprocessing import LabelEncoder

#fill in mean for floats
for c in df_most_frequent.columns:
    if df_most_frequent[c].dtype=='float16' or  df_most_frequent[c].dtype=='float32' or  df_most_frequent[c].dtype=='float64':
        df_most_frequent[c].fillna(df[c].mean())

#fill in -999 for categoricals
df_most_frequent = df_most_frequent.fillna(-999)
# Label Encoding
for f in df_most_frequent.columns:
    if df_most_frequent[f].dtype=='object': 
        lbl = LabelEncoder()
        lbl.fit(list(df_most_frequent[f].values))
        df_most_frequent[f] = lbl.transform(list(df_most_frequent[f].values))
        
print('Labelling done.')

In [ ]:
!pip install pycaret

#Pycaret has Regression and Classification

In [ ]:
from pycaret.regression import *

In [ ]:
#Code by Firat Gonen https://www.kaggle.com/frtgnn/pycaret-introduction-classification-regression

reg = setup(data = df_most_frequent, 
             target = 'engagement_index',
             numeric_imputation = 'mean',
             categorical_features = ['year','month','date','pct_access'], 
             #ignore_features = ['Alley'],
             normalize = True,
             silent = True)

In [ ]:
lightgbm = create_model('lightgbm')

#Tun the Model

In [ ]:
#Tun the model.

tuned_lightgbm = tune_model(lightgbm)

In [ ]:
plot_model(estimator = tuned_lightgbm, plot = 'learning')

#Feature Importance

In [ ]:
#Feature Importance

plot_model(estimator=tuned_lightgbm, plot='feature')

#Evaluate the Model

In [ ]:
evaluate_model(tuned_lightgbm)

#Decision Tree Regressor

In [ ]:
dt = create_model('dt')

In [ ]:
#bagged_dt = ensemble_model(dt, n_estimators=50)

In [ ]:
#boosted_dt = ensemble_model(dt, method = 'Boosting')

#Interpret Model

In [ ]:
interpret_model(tuned_lightgbm, plot = 'correlation')

#I made that to give a closure because that´s Not the kind of submission required in this Challenge. 

In [ ]:
#Make a prediction.

predictions = predict_model(tuned_lightgbm, data=df)
predictions.head()

In [ ]:
df['engagement_index'] = round(predictions['pct_access']).astype(int)
df.to_csv('submission.csv',index=False)
df.head()

#Code by Firat Gonen https://www.kaggle.com/frtgnn/pycaret-introduction-classification-regression

#Code by https://github.com/pycaret/pycaret/blob/master/examples/PyCaret%202%20Regression.ipynb